In [1]:
import torch as pt
from GDELTAnomalies.datasets.gdelt_pt_dataset import GDELTDataset

In [21]:
dataset = GDELTDataset(lookback=10, horizon=1, step=1, flatten=True, event_filter=["04", "13", "18", "19"])

In [42]:
train_len = 256
valid_len = 52
test_len = dataset.num_times - train_len - valid_len

valid_times = list(range(train_len, train_len + valid_len))
test_times = list(range(train_len + valid_len, dataset.num_times))

num_series = dataset.num_series

In [43]:
valid_preds = pt.zeros((len(valid_times), num_series), dtype=pt.double)
test_preds = pt.zeros((len(test_times), num_series), dtype=pt.double)

In [44]:
files = [f"checkpoints/TFT_ukr_small/TFT_ukr_small_preds_{i}.pt" for i in range(8)]
predictions = [pt.load(x) for x in files]

In [45]:
for prediction in predictions:
    v_preds = prediction["validation"]
    valid_idxs = prediction["validation_indices"]
    t_preds = prediction["test"]
    test_idxs = prediction["test_indices"]
    
    batches = len(v_preds)
    for batch in range(batches):
        valid_data = v_preds[batch].squeeze()
        test_data = t_preds[batch].squeeze()
        
        valid_series, valid_time = valid_idxs[batch]
        test_series, test_time = test_idxs[batch]
        valid_preds[valid_time - train_len, valid_series] = valid_data.to(dtype=pt.double)
        test_preds[test_time - train_len - valid_len, test_series] = test_data.to(dtype=pt.double)

In [46]:
preds = {"test_predictions": test_preds, "valid_predictions": valid_preds, "train_len": train_len, "valid_len": valid_len, "test_len": test_len, "colnames": dataset.columns}
pt.save(preds, "checkpoints/TFT_ukr_small/TFT_ukr_small_preds.pt")